In [2]:
import numpy as np 
import pandas as pd 
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, Model
from keras.callbacks import EarlyStopping,ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_ds="C:\Download/Birds525/train"
test_ds="C:\Download/Birds525/test"
validation_ds="C:\Download/Birds525/valid"

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255) #initialize train generator 
valid_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize validation generator 
test_datagen = ImageDataGenerator(rescale = 1.0/255.) #initialize test generator

In [5]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()
valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(train_ds,target_size=(224, 224),batch_size=32,shuffle=True,class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_ds,target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')
validation_generator = valid_datagen.flow_from_directory(validation_ds,target_size=(224,224),batch_size=32,shuffle=False,class_mode='categorical')

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [6]:
train_generator[0][0].shape

(32, 224, 224, 3)

In [7]:
print(len(train_generator))
print(len(test_ds))
print(len(validation_ds))

2645
25
26


In [8]:
# VGG 19 Model
class VGG19(Sequential):
    def __init__(self, input_shape):
        super().__init__()

        self.add(Conv2D(64, kernel_size=(3,3), padding= 'same', activation= 'relu', input_shape= input_shape))
        self.add(Conv2D(64, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
        
        self.add(Conv2D(128, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(128, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
        
        self.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(256, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
        
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))
        
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(Conv2D(512, kernel_size=(3,3), padding= 'same', activation= 'relu'))
        self.add(MaxPooling2D(pool_size=(2,2), strides= (2,2)))

        self.add(Flatten())
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dropout(0.5))
        self.add(Dense(4096, activation= 'relu'))
        self.add(Dropout(0.5))
        self.add(Dense(525, activation= 'softmax'))

        self.compile(optimizer= tf.keras.optimizers.Adam(0.003),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

In [9]:
model = VGG19((224, 224, 3))
model.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 64)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 112, 112, 128)     73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 112, 112, 128)     147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 128)      0         
 2D)                                                         

In [10]:
from keras.optimizers import Adam
import keras.losses
opt = Adam(learning_rate=0.00001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [11]:
history = model.fit(train_generator, validation_data = validation_generator, batch_size= 32, epochs=40)

Epoch 1/40
  60/2645 [..............................] - ETA: 13:14:45 - loss: 6.2686 - accuracy: 0.0016

KeyboardInterrupt: 

In [ ]:
n_categories = len(os.listdir("C:\Download/Birds525/train")) # number of categories print(n_categories)
n_categories

525

In [ ]:
results =pd.DataFrame(history.history)
results.head()

In [ ]:
plt.figure(figsize=(15,6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train','Val'], loc= 'upper left')
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
y_test_1 = test_generator.classes
y_pred_1 =model.predict(test_generator)
y_pred_1 = np.argmax(y_pred_1,axis=1)

results =model.evaluate(test_generator)